In [ ]:
!git clone https://github.com/ahmtkypnr/BBM416_AIN431_project

Cloning into 'BBM416_AIN431_project'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 50 (delta 21), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (50/50), 372.98 KiB | 1.99 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [ ]:
from BBM416_AIN431_project.data import create_shuffle_dataset,exclude_classes
import cv2
import os
from tqdm import tqdm
import shutil


In [ ]:
create_shuffle_dataset()

Dataset dizine aktarılıyor...


100%|██████████| 40156/40156 [07:46<00:00, 86.14it/s]


Dataset boyutu:  40156
/content/dataset/classes.json bulunamadı, yeni bir dosya oluşturulacak.


100%|██████████| 20078/20078 [00:05<00:00, 3716.71it/s]


/content/dataset/classes.json başarıyla oluşturuldu.
/content/dataset/images klasörü yok. (Sorun değil!)
/content/dataset/labels klasörü yok. (Sorun değil!)
txt'ler oluşturuluyor


...

100%|██████████| 16062/16062 [00:34<00:00, 468.42it/s]


val


100%|██████████| 2007/2007 [00:02<00:00, 782.15it/s]


test


100%|██████████| 2009/2009 [00:03<00:00, 649.25it/s]


In [ ]:
subsets = ["test"]

for subset in subsets:
    input_folder = f"dataset/images/{subset}"
    output_folder_bilinear = f"output_bilinear/{subset}"
    output_folder_lanczos = f"output_lanczos/{subset}"

    os.makedirs(output_folder_bilinear, exist_ok=True)
    os.makedirs(output_folder_lanczos, exist_ok=True)

    image_files = [f for f in os.listdir(input_folder) if f.endswith(".jpg") or f.endswith(".png")]

    print(f"\n🔄 {subset.upper()} klasörü işleniyor...")

    for filename in tqdm(image_files, desc=f"{subset}"):
        path = os.path.join(input_folder, filename)
        img = cv2.imread(path)
        if img is None:
            print(f"Hata: {filename} okunamadı.")
            continue

        # 1. Tüm görselleri 640x640'e normalize et
        base_img = cv2.resize(img, (640, 640), interpolation=cv2.INTER_AREA)

        # 2. 2 katına çıkar (1280x1280) – Bilinear
        bilinear_img = cv2.resize(base_img, (1280, 1280), interpolation=cv2.INTER_LINEAR)
        cv2.imwrite(os.path.join(output_folder_bilinear, filename), bilinear_img)

        # 3. 2 katına çıkar (1280x1280) – Lanczos
        lanczos_img = cv2.resize(base_img, (1280, 1280), interpolation=cv2.INTER_LANCZOS4)
        cv2.imwrite(os.path.join(output_folder_lanczos, filename), lanczos_img)

print("\n✅ Tüm subset'ler için SR (640→1280) işlemi tamamlandı.")


🔄 TEST klasörü işleniyor...


test: 100%|██████████| 2009/2009 [01:15<00:00, 26.77it/s]


✅ Tüm subset'ler için SR (640→1280) işlemi tamamlandı.


In [ ]:
subsets = ["test"]

for subset in subsets:
    src_label_dir = f"dataset/labels/{subset}"
    dst_label_bilinear = f"output_bilinear_labels/{subset}"
    dst_label_lanczos = f"output_lanczos_labels/{subset}"

    os.makedirs(dst_label_bilinear, exist_ok=True)
    os.makedirs(dst_label_lanczos, exist_ok=True)

    label_files = [f for f in os.listdir(src_label_dir) if f.endswith(".txt")]

    print(f"\n📄 {subset.upper()} etiketleri kopyalanıyor...")

    for filename in tqdm(label_files, desc=f"{subset} labels"):
        src_path = os.path.join(src_label_dir, filename)
        shutil.copy(src_path, os.path.join(dst_label_bilinear, filename))
        shutil.copy(src_path, os.path.join(dst_label_lanczos, filename))

print("\n✅ Tüm etiket kopyalama işlemi tamamlandı.")


📄 TEST etiketleri kopyalanıyor...


test labels: 100%|██████████| 2009/2009 [00:00<00:00, 6716.22it/s]


✅ Tüm etiket kopyalama işlemi tamamlandı.


In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ardaakgz","key":"aff4962cfb3cf18cb10ad8c35e1b9f44"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import json

metadata = {
    "title": "SR Dataset",
    "id": "ardaakgz/srDatasetFixed",
    "licenses": [{"name": "CC0-1.0"}]
}

with open("dataset-metadata.json", "w") as f:
    json.dump(metadata, f)

print("✅ dataset-metadata.json oluşturuldu.")

✅ dataset-metadata.json oluşturuldu.


In [ ]:
!zip -r output_bilinear.zip output_bilinear output_bilinear_labels

  adding: output_bilinear/ (stored 0%)
  adding: output_bilinear/test/ (stored 0%)
  adding: output_bilinear/test/775723355d35e76ca1d07a10eb04d857.jpg (deflated 2%)
  adding: output_bilinear/test/49980800d60b91710f7a070889f23901.jpg (deflated 2%)
  adding: output_bilinear/test/1b3b47e4d1b27a91d62cbb424d7ad0de.jpg (deflated 0%)
  adding: output_bilinear/test/20d0f2043b1c4bbb988b82953a751a29.jpg (deflated 0%)
  adding: output_bilinear/test/348432f303b3606d97764d394b655d2a.jpg (deflated 0%)
  adding: output_bilinear/test/ce611c986b86f19485d93c8b428d1598.jpg (deflated 0%)
  adding: output_bilinear/test/53e69be64c3cb611bb6b605c953e390c.jpg (deflated 0%)
  adding: output_bilinear/test/a677d677b18ac166c93ad4d8b3ed6d59.jpg (deflated 3%)
  adding: output_bilinear/test/fdd754fd04aa0c8fdefdbb39b8206445.jpg (deflated 0%)
  adding: output_bilinear/test/c4234fd617071f418e94c7bda9f2ecc5.jpg (deflated 0%)
  adding: output_bilinear/test/de3b6a24d45c0da6cc4b7888579d9c3e.jpg (deflated 0%)
  adding: outpu

In [ ]:
!zip -r output_lanczos.zip output_lanczos output_lanczos_labels

  adding: output_lanczos/ (stored 0%)
  adding: output_lanczos/test/ (stored 0%)
  adding: output_lanczos/test/775723355d35e76ca1d07a10eb04d857.jpg (deflated 2%)
  adding: output_lanczos/test/49980800d60b91710f7a070889f23901.jpg (deflated 2%)
  adding: output_lanczos/test/1b3b47e4d1b27a91d62cbb424d7ad0de.jpg (deflated 0%)
  adding: output_lanczos/test/20d0f2043b1c4bbb988b82953a751a29.jpg (deflated 0%)
  adding: output_lanczos/test/348432f303b3606d97764d394b655d2a.jpg (deflated 0%)
  adding: output_lanczos/test/ce611c986b86f19485d93c8b428d1598.jpg (deflated 0%)
  adding: output_lanczos/test/53e69be64c3cb611bb6b605c953e390c.jpg (deflated 0%)
  adding: output_lanczos/test/a677d677b18ac166c93ad4d8b3ed6d59.jpg (deflated 3%)
  adding: output_lanczos/test/fdd754fd04aa0c8fdefdbb39b8206445.jpg (deflated 0%)
  adding: output_lanczos/test/c4234fd617071f418e94c7bda9f2ecc5.jpg (deflated 0%)
  adding: output_lanczos/test/de3b6a24d45c0da6cc4b7888579d9c3e.jpg (deflated 0%)
  adding: output_lanczos/tes

In [ ]:
!kaggle datasets create -p .

Skipping folder: .config; use '--dir-mode' to upload folders
Skipping folder: output_bilinear_labels; use '--dir-mode' to upload folders
Skipping folder: output_lanczos; use '--dir-mode' to upload folders
Skipping folder: output_bilinear; use '--dir-mode' to upload folders
Skipping folder: output_lanczos_labels; use '--dir-mode' to upload folders
Starting upload for file output_bilinear.zip
100% 501M/501M [00:18<00:00, 29.1MB/s]
Upload successful: output_bilinear.zip (501MB)
Starting upload for file output_lanczos.zip
100% 568M/568M [00:20<00:00, 29.6MB/s]
Upload successful: output_lanczos.zip (568MB)
Skipping folder: BBM416_AIN431_project; use '--dir-mode' to upload folders
Starting upload for file kaggle.json
100% 64.0/64.0 [00:00<00:00, 84.5B/s]
Upload successful: kaggle.json (64B)
Skipping folder: dataset; use '--dir-mode' to upload folders
Skipping folder: sample_data; use '--dir-mode' to upload folders
Your private Dataset is being created. Please check progress at https://www.ka